# Cifar-100 이미지 분류 - ver.2
- Cifar-100용 별도의 모델
- 32/64/128 Conv2D node 수
- Adam() optimizer 사용

In [ ]:
import numpy as np 
import tensorflow as tf
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

- 상수 설정

In [ ]:
BATCH_SIZE = 32
NUM_CLASSES = 100
EPOCHS = 100
MODEL_PATH = 'model/best_cifar-100.h5'

In [ ]:
CIFAR100_LABELS = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]

## 데이터 전처리

In [ ]:
from tensorflow.keras.datasets import cifar100
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

In [ ]:
X_train.shape, X_test.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.subplot(141)
plt.imshow(X_train[0], interpolation='bicubic')
plt.subplot(142)
plt.imshow(X_train[4], interpolation='bicubic')
plt.subplot(143)
plt.imshow(X_train[8], interpolation='bicubic')
plt.subplot(144)
plt.imshow(X_train[12], interpolation='bicubic')
plt.show()

In [ ]:
CIFAR100_LABELS[y_train[0][0]], CIFAR100_LABELS[y_train[4][0]], \
    CIFAR100_LABELS[y_train[8][0]], CIFAR100_LABELS[y_train[12][0]]

In [ ]:
# 0~1 사이로 정규화
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# One-hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)
Y_train.shape, Y_test.shape

## 모델 정의/설정/학습/평가

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
model = Sequential([ 
    Conv2D(32, (3,3), padding='same', input_shape=(32,32,3)),
    Activation('relu'),
    MaxPooling2D(),
    Activation('relu'),

    Conv2D(64, (3,3)),
    Activation('relu'),
    MaxPooling2D(),
    Activation('relu'),
    Dropout(0.1),

    Conv2D(128, (3,3), padding='same'),
    Activation('relu'),
    MaxPooling2D(),
    Activation('relu'),
    Dropout(0.25), 

    Flatten(),
    Dense(512),
    Activation('relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')        
])
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.0001)

model.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:

import os
if not os.path.exists('model'):
    os.mkdir('model')
checkpointer = ModelCheckpoint(
    MODEL_PATH, verbose=1, save_best_only=True
)
early_stopping = EarlyStopping(patience=10)

In [ ]:
history = model.fit(
    X_train, Y_train, validation_split=0.2,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[checkpointer, early_stopping]
)

In [ ]:

best_model = load_model(MODEL_PATH)
best_model.evaluate(X_test, Y_test)